In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

experiments = pd.read_parquet("results_iterations.parquet")
experiments["error"] = np.abs(experiments["estimate"])
experiments["seed"] = 96*[i for i in range(10000)]

range_ns = np.unique(experiments["n"])
range_deltas = -np.sort(-np.unique(experiments["delta"]))
range_distributions = np.unique(experiments["distribution"])

# organize results
df_grouped = [ experiments[ experiments["delta"] == delta ].groupby(["n", "delta", "method", "distribution"])["error"].quantile(1-delta).reset_index() for delta in range_deltas ]
df_grouped = pd.concat(df_grouped)

df_grouped_pivot = df_grouped.pivot(index=['n', 'delta', 'distribution'], columns='method', values='error').reset_index()

for rho in ["atm", "win", "lv"]:
    df_grouped_pivot[f"tm {rho}"] = (df_grouped_pivot[f"tm {rho}"] -  df_grouped_pivot["tm"])/df_grouped_pivot["tm"]
    df_grouped_pivot[f"mean {rho}"] = (df_grouped_pivot[f"mean {rho}"] -  df_grouped_pivot["mean"])/df_grouped_pivot["mean"]

del df_grouped_pivot["mean"]
del df_grouped_pivot["tm"]

df_grouped = df_grouped_pivot.melt(id_vars=['n', 'delta', 'distribution'], 
                                    var_name='baseline', 
                                    value_name='change')

df_grouped

,n,delta,distribution,baseline,change
0,50,0.001,Gaussian,mean atm,0.078561
1,50,0.001,St df=2.01,mean atm,-0.786565
2,50,0.010,Gaussian,mean atm,0.062052
3,50,0.010,St df=2.01,mean atm,-0.645791
4,50,0.100,Gaussian,mean atm,0.091578
...,...,...,...,...,...
67,500,0.001,St df=2.01,tm win,0.043512
68,500,0.010,Gaussian,tm win,0.001212
69,500,0.010,St df=2.01,tm win,-0.046556
70,500,0.100,Gaussian,tm win,-0.001748


In [57]:
for distribution in range_distributions:
    print(distribution)
    for rho in ["lv", "atm", "win"]:
        line = rho
        for n in range_ns:
            for delta in range_deltas:
                for base in ["tm", "mean"]:
                    change = df_grouped.change[ (df_grouped.n == n) & (df_grouped.delta == delta) & (df_grouped.delta == delta) & (df_grouped.distribution == distribution) & (df_grouped.baseline == f"{base} {rho}") ].values[0]
                    line += f"& {change*100:.0f}"
        print(line)


Gaussian
lv& -0& 1& 1& 1& 1& 2& -0& 0& 1& 0& -1& -0
atm& 9& 9& 8& 6& 11& 8& 11& 12& 11& 10& -1& 0
win& 1& 1& 2& 1& 0& 2& -0& 0& 0& -0& 2& 3
St df=2.01
lv& 51& -11& 179& -12& 475& -35& 47& -2& 100& -2& 247& 0
atm& -0& -41& 3& -65& 7& -79& -2& -34& -3& -53& 4& -70
win& 1& -35& 0& -56& 2& -60& -1& -34& -5& -53& 4& -70
